Pandas非常善于处理大规模数据，支持将结果导出到CSV，Excel，HTML，json等文件中。 如果您想将 ``多种信息（excel、图片等）组合`` 到一个文档中，那么事情变得更加困难。 例如，如果要在一个Excel工作表上放置两个DataFrame，则需要使用相应的Excel库才能输出。 这当然是OK的，但走起来真不简单。 本文将介绍一种方法，将多种 ``信息（excel、图片等）组合`` 成一个 ``HTML模板`` ，然后使用``Jinja模板和WeasyPrint`` 再将其转换为独立的PDF文档。

## 操作流程
使用Pandas将数据输出到Excel文件中的多个工作表或者用DataFrames创建多个Excel文件非常方便。 但是，如果您想将 ``多种信息（excel、图片等）组合`` 输出到一个文件中，那么直接从Pandas进行操作的方法并不多。 幸运的是，万能的python大法有很多选项可以帮助我们。

在本文中，我将采用以下流程来创建多页PDF文档。
![](img/pdf-conversion-pipeline.png)

## 工具
我决定使用HTML作为模板语言，因为它可能是生成结构化数据最便捷的方法，支持各种格式数据（如图片、excel表）。我也认为每个人都知道（或可以弄清楚）足够的HTML知识来生成一个简单的报告。

这个工作流程中最困难的部分是弄清楚如何将HTML呈现为PDF。我觉得还没有最佳的解决方案，但我选择了 ``WeasyPrint`` ，因为它仍然在积极维护，我发现我可以相对容易地使它工作。作为替代方案，我过去使用过xhtml2pdf，它的效果也很好，遗憾的是该库缺乏文档说明，但它已存在一段时间，并且确实可以从HTML生成PDF。

## 数据
如上所述，我们将使用之前文章中的相同数据集。 为了使这一切成为一个独立的文章，下面是我如何导入数据,做描述性统计，及并生成关于CPU和软件销售的 ``数据透视表`` 。

导入模块，并读入销售渠道信息。

In [1]:
import pandas as pd
import numpy as np

df = pd.read_excel('salesfunnel.xlsx')
df.head()

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


对数据做透视表

In [2]:
sales_report = pd.pivot_table(df, index=["Manager", "Rep", "Product"], 
                              values=["Price", "Quantity"],
                              aggfunc=[np.sum, np.mean], 
                              fill_value=0)
sales_report.head()

sum            mean         
                                         Price Quantity  Price Quantity
Manager      Rep           Product                                     
Debra Henley Craig Booker  CPU           65000        2  32500      1.0
                           Maintenance    5000        2   5000      2.0
                           Software      10000        1  10000      1.0
             Daniel Hilton CPU          105000        4  52500      2.0
                           Software      10000        1  10000      1.0

## 模板
DataFrame拥有to_html()的方法，可以将dataframe数据转化为含有HTML格式的字符串。

但是随着您的报告变得越来越复杂或您选择将Jinja模板用于您的网络应用，jinja的这些功能将为您提供良好的服务。Jinja的另一个不错的功能是它包含多个内置过滤器，这些过滤器允许我们以Pandas中难以做到的方式格式化我们的一些数据。

为了在我们的应用程序中使用Jinja，我们需要做三件事：

```
1. 创建一个模板
2. 将变量添加到模板上下文中
3. 将模板渲染为HTML
```

这是一个非常简单的模板，我们称之为myreport.html：

```html
<!DOCTYPE html>
<html>
<head lang="en">
    <meta charset="UTF-8">
    <title>{{ title }}</title>
</head>
<body>
    <h2>Sales Funnel Report - National</h2>
     {{ national_pivot_table }}
</body>
</html>
```

上面html代码中拥有 ``{{title}} 和 {{national_pivot_table }}`` 这两个关键词。这两个关键词用来接收需要渲染的数据，比如图片对象、dataframe对象等。

要填充这些变量，我们需要创建一个Jinja环境并获取我们的模板对象：

In [3]:
from jinja2 import Environment, FileSystemLoader
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template('myreport.html')

在上面的示例中，我假设模板位于当前目录中，但您可以将完整路径放到模板位置。

另一个关键组件是 ``env`` 的创建。 这个变量是我们将数据（或对象）填充给模板的方式。 我们创建了一个名为 ``template_var`` 的字典，其中包含我们想要传递给模板的所有变量。

``请注意变量的名称如何与我们的模板匹配。即名称要与myreport.html中的{{}}夹住的关键词一样。``

In [4]:
template_vars = {"title" : "Sales Funnel Report - National",
                 "national_pivot_table": sales_report.to_html()}

最后一步是将 ``template_vars`` 渲染到html模板中，并输出为html字符串，最终我们将使用该html字符串来生成pdf报告。 

In [5]:
html_out = template.render(template_vars)

为简洁起见，我不会显示完整的HTML，但您应该明白这一点。
## 生成pdf
pdf报告的生成部分相当简单，这里用到weasyprint库

In [9]:
from weasyprint import HTML
HTML(string=html_out).write_pdf("report.pdf")

![](img/ugly_table.png)
但是打开的pdf样式很简单，白底黑字并不美观。之所以这样是因为我们没有使用自定义样式表css，但是咱们不熟悉css，有一种简单的办法就是用成熟的css，这里我用的``typography.css`` 作为填充表格时的样式表。这个css的优点有：

```
1. 相对较小且易于理解
2. 可以在PDF引擎中运行而不会抛出错误和警告
3. 能让表格表格看起来很美观
```

让我们尝试使用我们更新的样式表重新渲染它：

In [12]:
HTML(string=html_out).write_pdf('beautiful_report.pdf', stylesheets=["typography.css"])

![](img/beautiful_table.png)
仅仅是添加了一个stylesheets参数就让输出结果瞬间变得高端大气上档次。

## 更复杂的模板
到目前为止，我们输出的pdf结果中只有一种数据，即dataframe。如果pdf中除了dataframe信息，我们还要有excel信息，这就要用到to_excel()方法。

为了生成更有用的报告，我们将结合上面显示的摘要统计信息以及分析报告，以便为每个Manager包含一个单独的PDF页面。

让我们从更新的模板（newmyreport.html）开始：

```html
<!DOCTYPE html>
<html>
<head lang="en">
    <meta charset="UTF-8">
    <title>{{ title }} </title>
</head>
<body>
<div class="container">
    <h2>Sales Funnel Report - National</h2>
     {{ national_pivot_table }}
    {% include "summary.html" %}
</div>
<div class="container">
    {% for manager in Manager_Detail %}
        <p style="page-break-before: always" ></p>
        <h2>Sales Funnel Report - {{manager.0}}</h2>
        {{manager.1}}
        {% include "summary.html" %}
    {% endfor %}
</div>
</body>
</html>
```

你会注意到的第一件事是有一个include语句提到了另一个文件。 include允许我们引入一段HTML代码并在代码的不同部分中重复使用它。在这种情况下，摘要包含我们希望包含在每个报告中的一些简单的国家级统计数据，以便管理人员可以将其绩效与全国平均水平进行比较。

这是summary.html的样子：

```html
<h3>National Summary: CPUs</h3>
    <ul>
        <li>Average Quantity: {{CPU.0|round(1)}}</li>
        <li>Average Price: {{CPU.1|round(1)}}</li>
    </ul>
<h3>National Summary: Software</h3>
    <ul>
        <li>Average Quantity: {{Software.0|round(1)}}</li>
        <li>Average Price: {{Software.1|round(1)}}</li>
    </ul>
```

在这个片段中，您将看到我们可以访问的其他变量：``CPU和Software``。其中每个都是``一个python列表``，其中包括CPU和Software销售的平均数量和价格。

您可能还注意到我们使用了 ``|`` 将每个值舍入到小数点后1位。这是使用Jinja过滤器的一个具体示例。

还有一个 ``for循环`` ，允许我们在报告中显示每个 ``Manager``的详细信息。 Jinja的模板语言只包含一小部分改变控制流的代码。基本for循环几乎是任何模板的支柱，因此它们应该对大多数人有意义。

我想调出一段看起来有点不合适的最后一段代码：

```
<p style="page-break-before: always" ></p>
```

这是一个简单的CSS指令，加了这个样式的元素，会和后面的隔开，单独作为一页，该元素后的内容被作为另起一页。

## 额外的统计量
现在我们已经完成了模板，这里是如何创建模板中使用的额外变量。

这是一个简单的汇总函数：

In [21]:
def get_summary_stats(df, product):
    """
    For certain products we want National Summary level information on the reports
    Return a list of the average quantity and price
    """
    results = []
    results.append(df[df["Product"]==product]["Quantity"].mean())
    results.append(df[df["Product"]==product]["Price"].mean())
    return results

我们也需要创建manager details 

In [22]:
manager_df = []
for manager in sales_report.index.get_level_values(0).unique():
    manager_df.append([manager, sales_report.xs(manager, level=0).to_html()])

最后，在模板中调用这些变量

In [23]:
template_vars2 = {"title" : "National Sales Funnel Report",
                 "CPU" : get_summary_stats(df, "CPU"),
                 "Software": get_summary_stats(df, "Software"),
                 "national_pivot_table": sales_report.to_html(),
                 "Manager_Detail": manager_df}

# Render our file and create the PDF using our css style file
html_out = template.render(template_vars2)
HTML(string=html_out).write_pdf("wholereport.pdf",  stylesheets=["typography.css"])

In [20]:
get_summary_stats(df, "CPU")

[1.8888888888888888, 51666.666666666664]

pip3 install cairocffi
brew install cairo
brew install pango